<a href="https://colab.research.google.com/github/sibat119/papers-review-code-impl/blob/main/word2vec/word2vec_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>